# Question Answering over Resources

## Import Dependencies 

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chat_models import ChatOpenAI
import os
import getpass 
from bardapi import Bard
from langchain.document_loaders import PyPDFLoader
import openai

## Initialize OpenAI & Bard

In [2]:
bard_token = getpass.getpass("Enter the token for Google Bard: ")
bard = Bard(token=bard_token)

openai_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ['OPENAI_API_KEY'] = openai_key
embeddings = OpenAIEmbeddings()

## Read Files

### TXT Files

In [3]:
with open("Test-Documents/state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
pages = text_splitter.split_text(state_of_the_union)
db = Chroma.from_texts(pages, embeddings, metadatas=[{"source": str(i)} for i in range(len(pages))])

### PDF Files

In [20]:
loader = PyPDFLoader("Test-Documents/wsj.pdf")
pages = loader.load_and_split()
db = Chroma.from_documents(pages, embeddings)

### CSV Files

In [23]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='Test-Documents/Passwords.csv')
data = loader.load()
db = Chroma.from_documents(data, embeddings)

### HTML Files

In [38]:
from langchain.document_loaders import UnstructuredHTMLLoader

loader = UnstructuredHTMLLoader("nature.html")
data = loader.load()
db = Chroma.from_documents(data, embeddings)

### Markdown Files

In [ ]:
from langchain.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader("INPUT FILE PATH")
data = loader.load()
db = Chroma.from_documents(data, embeddings)

## Ask Questions

In [4]:
query = "What did president say about China?"
docs = db.similarity_search(query)

para = ""
for each in docs: 
    para += each.page_content

In [5]:
model = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)
chain = load_qa_with_sources_chain(model, chain_type="map_reduce")
openai_res = chain({"input_documents": docs, "question": query}, return_only_outputs=True)

prompt = f"I want you to work as a question answering machine over given documents. The documents you will be given is the following paragraph: {para}. Then, I want to you answer the question based on the given paragraph. The question is {query}"
bard_res = bard.get_answer(prompt)['content']

In [7]:
print(openai_res['output_text'] + "\n")
print(bard_res)

The president mentioned that we need to level the playing field with China and other competitors in order to compete for the best jobs of the future.
SOURCES: 12

In the given paragraph, President Biden said that he told Xi Jinping that it is never a good bet to bet against the American people. He also said that the United States needs to level the playing field with China and other competitors in order to compete for the best jobs of the future. He specifically mentioned the Bipartisan Innovation Act, which would make record investments in emerging technologies and American manufacturing.

Here are some specific quotes from the paragraph that refer to China:

* "As I’ve told Xi Jinping, it is never a good bet to bet against the American people."
* "But to compete for the best jobs of the future, we also need to level the playing field with China and other competitors."
* "That’s why it is so important to pass the Bipartisan Innovation Act sitting in Congress that will make record inve

## Language Translation Support

In [ ]:
language = "Spanish"
translated = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": f"You are a {language} translator."},
        {"role": "user", "content": f"I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in {language}. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level f{language} words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements and nothing else, do not write explanations. The paragrah you will translate is {bard_res}."}
    ]
)

print(translated["choices"][0]["message"]["content"])

## Highlight Original Sources